How to use gulp?

In [ ]:
var gulp = require('gulp');
var tap = require('gulp-tap');
var replace = require('gulp-replace');

// rename the project
gulp.task('build :src :dest', function (files, project) {
    return gulp
        .src(files, {cwd: project})
        .pipe(replace('appId: \'my-app-id\'', 'appId: \'jupytangular-module-server\''))
        .pipe(tap(function (file) {
            console.log(file.contents.toString());
        }))
});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = PROFILE_PATH + '/Documents/universal';

gulp.task('default', () => gulp.task(['build :src :dest'])(['**/app.browser.module.ts'], project));

$$.async()
try {
    gulp.series(() => gulp.task(['default'])(), function (done) {
        done();
        $$.sendResult();
    })();
} catch (e) {
    $$.sendError(e);
}

How to convert a gulp task to a Promise?

How to use a Promise as a gulp task?



In [ ]:
var gulp = require('gulp');

// I wish Linus Torvolds had absolute power over everything javascript(+derivatives) and he could just do a massive purge of all the bullshit.
function tasksToPromise(tasks) {
    return new Promise((resolve, reject) => {
        gulp.series.apply(gulp, tasks.concat([function (done) {
            resolve(done());
        }]))();
    });
};

module.exports = tasksToPromise;



In [ ]:
var fs = require('fs');
var path = require('path');
var replace = require('gulp-replace');
var gulp = require('gulp');

// add automock to project for callback services
var client, renderer, server, core;

function applyUniversal(project) {
    // gulp.task('theme',
    //    gulp.src('./some/dir/**/*.js', {cwd: 'path/to/project'})
    //        .src('./specific-file.ts')) // already within cwd in chain?

    // TODO: copy auth, resolve dependencies using file system script, generate code below

    // replace package.json
    gulp.task(
        'project relatives',
        gulp.series(projectRelatives(project)))
    // in *.js Can't resolve 'socket.io-client/package'
    // in *.ts Cannot find name 'JwtHelper' -> trace back to import statements?


    gulp.task('server module', function () {
        return gulp.src(['**/server/app/app.server.module.js'], {cwd: project})
        // add mock socket server to server/render-server or app.server.module
            .pipe(replace('import { AppComponent } from \'./app.component\'', `
import { AppComponent } from './app.component';
import { AuthService } from '../../client/app/auth/auth.service';
import { LogService } from '../../imports/log.service';
import { AuthManager } from '../../client/app/auth/auth.manager';
import { JwtHelper } from 'angular2-jwt/angular2-jwt';
import { MailgunValidatorService } from '../../client/app/auth/mailgun-validate.service';
`))
            // TODO: pipe server.ts to add log provider for log service
            .pipe(replace(
                /\)\],\s*AppModule\s*,providers\s*:\s*\[/ig,
                `)],
        AppModule
    ],
    providers: [
        {
            provide: AuthService,
            useValue: require('./sockify-server').sockifyRequire(AuthService, 'AuthService')
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-server').sockifyRequire(PlatformLocation, 'PlatformLocation')
        },
        {
            provide: LogService,
            useValue: require('./sockify-server').sockifyRequire(LogService, 'LogService')
        },
        {
            provide: AuthGuard,
            useValue: require('./sockify-server').sockifyRequire(AuthGuard, 'AuthGuard')
        },
        AuthManager, JwtHelper, MailgunValidatorService,

    `))
            .pipe(gulp.dest(project))
    });

    // add socket-client provider for auth module to app.browser.module
    gulp.task('browser module', function () {
        return gulp.src(['**/client/app/app.browser.module.ts'], {cwd: project})
            .pipe(replace('import { AppComponent } from \'./app.component\'', `
import { AppComponent } from './app.component';
import { AuthService } from './auth/auth.service';
import { PlatformLocation } from '@angular/common';
import { LogService } from '../../imports/log.service';
import { AuthManager } from './auth/auth.manager';
import { JwtHelper } from 'angular2-jwt/angular2-jwt';
import { MailgunValidatorService } from './auth/mailgun-validate.service';
`))
            .pipe(replace(
                /\)\],\s*AppModule/ig,
                `)],
        AppModule,
    ],
    providers: [ AuthManager, JwtHelper, MailgunValidatorService,
        {
            provide: AuthService,
            useValue: require('./sockify-client').sockifyClient(AuthService, 'AuthService'),
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-client').sockifyClient(PlatformLocation, 'PlatformLocation'),
        },
`))
            .pipe(gulp.dest(project))
    });

    // replace routes with lazy modules from other projects
    gulp.task('app router', function () {
        return gulp.src(['**/client/app/app.routes.ts'], {cwd: project})
            .pipe(replace(
                /children: \[/ig,
                `children: [
            {
                path: 'auth',
                loadChildren: './auth/auth.module#AuthModule'
            },
`))
            .pipe(gulp.dest(project))
    });

    // TODO: try new versions of packages to act like greenkeeper.io

    // gulp apply everything
    gulp.task('apply universal',
        gulp.series('server module', 'browser module''app router'));

    return tasksToPromise(['apply universal'], []);
};
applyUniversal;



Apply universal to angular project?

In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = PROFILE_PATH + '/Documents/universal';
if (!fs.existsSync(project)) {
    fs.mkdirSync(project);
}

$$.async()
applyUniversal(project)
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Fix project relatives as a service for updating files when they change?



In [ ]:
var gulp = require('gulp'),
    watch = require('gulp-watch');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = PROFILE_PATH + '/Documents/universal';

var searching = false;

gulp.task('watch for changes', function () {
    // Callback mode, useful if any plugin in the pipeline depends on the `end`/`flush` event 
    return watch('src/**/*.ts', {
        cwd: project,
        ignoreInitial: true,
        read: false,
        readDelay: 100
    }, function () {
        if (!searching) {
            searching = true;
            projectRelatives(project).then(() => {
                searching = false
            });
        }
    });
});

$$.async();
gulp.task(['watch for changes'])();



Search notebooks for a string using gulp?

search notebooks gulp?



In [ ]:
var importer = require('../Core');
var path = require('path');
var gulp = require('gulp');
var tap = require('gulp-tap');
var Duplex = require('stream').Duplex;
var tasksToPromise = importer.import('gulp tasks to promise', {gulp});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(__dirname, '..');

function bufferToStream(buffer) {
    let stream = new Duplex();
    stream.push(buffer);
    stream.push(null);
    return stream;
}

function searchNotebooks(search) {
    var cells;
    gulp.task('search notebooks', function () {
        cells = [];
        return gulp.src(
            ['**/*.ipynb', '!node_modules{,/**}', '!**/\\.*'],
            {cwd: project})
            .pipe(tap(file => {
                const result = importer.getCells(file.path);
                result.forEach(r => {
                    if(r.source.join('').match(search)) {
                        cells.push(r);
                    }
                })
            }));
    });

    return tasksToPromise(['search notebooks'], [])
        .then(() => cells);
}

module.exports = searchNotebooks;



In [ ]:
var importer = require('../Core');
var searchNotebooks = importer.import('search notebooks using gulp');
var searchRegexp = 'permissions';

$$.async();
searchNotebooks(searchRegexp)
    .then(r => $$.mime({'text/markdown': '```\n' + r.map(c => c.code).join('') + '\n```\n-------------\n'}))
    .catch(e => $$.sendError(e));


Find relative paths and includes?

In [ ]:
var fs = require('fs');
var path = require('path');
var gulp = require('gulp');
var tap = require('gulp-tap');
var importer = require('../Core');
var taskToPromise = importer.import('gulp.ipynb[Promise gulp task]', {gulp});
var getRequires = importer.import('get requires from source');

var relatives = [];
var packages = [];
var missing = [];
var missingPackages = [];

gulp.task('get relative imports', function () {
    return gulp.src(['**/*.+(ts|js|ipynb)'], {
        ignore: ['**/node_modules/**',
                '**/packages/**',
                '**/vendor/**',
                '/Users/**/Downloads/**',
                '/Users/**/Library/**',
                '/Users/**/Applications/**',
                '/Users/**/Music/**',
                '**/wp-content/**',
                '**/wp-includes/**',
                '**/Pods/**',
                '**/svn/**',
                '**/.git/**',
                '**/.vscode/**',
                '**/.npm/**',
                '**/\\.*',
                '**/.*',
                '**/Cache/**',
                '**/Creative Cloud Files/**'],
        cwd: project
    })
    .pipe(tap(function (file) {
        console.log(`Scanning ${file.path}`);
        var requires = getRequires(file.contents.toString());
        requires.forEach(path => {
            var event = {file: file.path, import: path};
            var assignee = path.match(/^\./) ? relatives : packages;
            assignee[assignee.length] = event;
            try {
                require.resolve(path);
            } catch (e) {
                var missingList = path.match(/^\./) ? missing : missingPackages;
                if (!missingList.includes(path)) {
                    missingList[missingList.length] = event;
                }
            } finally {
            }
        })
    }))
})

function getRelativeImports(p) {
    relatives = [];
    packages = [];
    missing = [];
    missingPackages = [];
    project = p;
    return taskToPromise(['get relative imports'])
        .then(r => ({relatives, packages, missing, missingPackages}))
        .catch(e => console.log(e))
};
module.exports = getRelativeImports;


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/universal') : project;

var importer = require('../Core');
var projectRelatives = importer.import('fix project paths');


$$.async();
projectRelatives(project)
    //.then(r => console.log(r))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));

